In [18]:
# core scientific libraries
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import datetime

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# secondary libraries
import netCDF4 as nc
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, interplevel, CoordPair, vertcross, g_uvmet, interpline, destagger,
                interp2dxy, ll_to_xy, xy, ALL_TIMES, vinterp, uvmet)
import metpy.calc as mpcalc
from metpy.units import units
import metpy.constants as constants
from math import fabs, log, cos, sin, tan, pi

import pytz

from xarray.backends.netCDF4_ import NetCDF4DataStore

import copy 
import time
from datetime import datetime

In [19]:
# Set filepath
# datdir = '../input_data/'
# filename = 'wrfout_d03_2017-05-25_00:00:00'

datdir = '/home/sundowners/non-sundowner/2017/'
filename = 'wrfout_d04_2017-05-25_00:00:00'

# set output filepath
outdir = '/home/sbarc/students/coello/repos/classes/spring_2021/geog_288cj/post_processing/outputs/'
fig_outdir = '/home/sbarc/students/coello/repos/classes/spring_2021/geog_288cj/post_processing/outputs/figures/'
dat_outdir = '/home/sbarc/students/coello/repos/classes/spring_2021/geog_288cj/post_processing/outputs/data/'


# open wrf Dataset object
filepath = datdir + filename
print('Opened File:', filepath, '\n')

# Open NetCDF Dataset object
f = nc.Dataset(filepath,'r')

Opened File: /home/sundowners/non-sundowner/2017/wrfout_d04_2017-05-25_00:00:00 



In [20]:
# specify desired variables to grad from the wrf out file and their units
des_vars = ["T2", "slp", "uvmet10", "uvmet10_wspd_wdir", "rh2"]
var_units = [None, "hpa", "m s-1", "m s-1", None]

# save the number of times in the file
n_times = len(f['Times'])

# make empty list to append interpolated dataarrays too
data = []

# start timer
t0 = time.time()

# loop through variables
for i in range(len(des_vars)): 
    
    # make empty list to append getvar single index times too
    tmp_list = []
    tmp_list2 = []
    
    # loop through times
    for indx in range(n_times):
    
        # grab var with units at time index
        if (var_units[i] != None):
            tmp_indx = getvar(f, des_vars[i], units = var_units[i], timeidx = indx)
        else:
            tmp_indx = getvar(f, des_vars[i], timeidx = indx)
        
        if (des_vars[i] == "uvmet10" or des_vars[i] == "uvmet10_wspd_wdir"): # some get vars output two variables
            tmp_1, tmp_2 = tmp_indx

            if (des_vars[i] == "uvmet10"):
                tmp_1 = tmp_1.rename('u').drop("u_v")
                tmp_2 = tmp_2.rename('v').drop("u_v")
            else:
                tmp_1 = tmp_1.rename('wspd').drop("wspd_wdir")
                tmp_2 = tmp_2.rename('wdir').drop("wspd_wdir")

            tmp_list.append(tmp_1)
            tmp_list2.append(tmp_2)

        else:
            
            tmp_list.append(tmp_indx)

    
    if (des_vars[i] == "uvmet10" or des_vars[i] == "uvmet10_wspd_wdir"): # some get vars output two variables
    
        # combine all timesteps together
        tmp = xr.concat(tmp_list, dim = 'time')
        tmp2 = xr.concat(tmp_list2, dim = 'time')

    
        # fix projection labeling for saving as a netcdf https://github.com/NCAR/wrf-python/issues/91
        del tmp.attrs['coordinates']
        del tmp2.attrs['coordinates']

        tmp.attrs['projection'] = str(tmp.attrs['projection'])
        tmp2.attrs['projection'] = str(tmp2.attrs['projection'])

        # append to list for all variables
        data.append(tmp)
        data.append(tmp2)
    
    
    else:
        
        # combine all timesteps together
        tmp = xr.concat(tmp_list, dim = 'time')

        # fix projection labeling for saving as a netcdf https://github.com/NCAR/wrf-python/issues/91
        del tmp.attrs['coordinates']

        tmp.attrs['projection'] = str(tmp.attrs['projection'])

        # append to list for all variables
        data.append(tmp)
    
    
# merge into one dataset    
data = xr.merge(data)

# add terrian height to dataset after correcting the projection labeling
terr = getvar(f, "ter", units = 'm', timeidx = 0)

del terr.attrs['coordinates']

terr.attrs['projection'] = str(terr.attrs['projection'])
 
data['terr']  = terr  
    
# pull lat and lon vallues and replace current corrdinates
lats = data.XLAT.values[:,0]
lons = data.XLONG.values[0,:]

data['south_north'] = lats
data['west_east'] = lons

# remane dimensions with ugly names
data = data.rename({'south_north':'lat', 'west_east':'lon', 'XTIME':'time', 'XLAT':'lat_grid', 
                    'XLONG':'lon_grid'})

# fix the time coordinates
data['time'] = data['Time']
data = data.drop("Time")

# save time variable
timez = data.time.values

# save times as datetimes
dates = []
dates_list = []

# store dates as a datetine and convert it to PDT timezone
for i in range(len(timez)):
    dates_obj = datetime.strptime(str(timez[i]), '%Y-%m-%dT%H:%M:%S.000000000')
    dates_obj_zone = dates_obj.replace(tzinfo =  pytz.timezone('UTC'))
    dates_obj_PDT = dates_obj_zone.astimezone(pytz.timezone('US/Pacific'))
    datetime64_PDT = np.datetime64(str(dates_obj_PDT)[:16]).astype(datetime)
    dates.append(datetime64_PDT)

    dates_list.append(datetime64_PDT)

data["time"] = dates_list

# end timer
t1 = time.time()
total = t1-t0
print('----------------------')
print("time:", total)

----------------------
time: 53.81757712364197


In [21]:
# convert temp from K to c
data['T2'] = data['T2'] - 273.15

In [22]:
filename_out = filename + "_surf" #+ "_climo"

data.to_netcdf(path = dat_outdir + filename_out)

print('Saved Files As: \n', dat_outdir + filename_out)

Saved Files As: 
 /home/sbarc/students/coello/repos/classes/spring_2021/geog_288cj/post_processing/outputs/data/wrfout_d04_2017-05-25_00:00:00_surf_climo
